# PyTorch Lightning fine-tuning template

by Andrés Muñoz-Jaramillo

This notebook is meant to act as a template to train and use a surya model to implement DS application.

It focuses on the concept of defining a modified Surya model, loading its weigths, and using a PyTorch lightning training loop to train it

This notebook assumes familiarity with the concepts of datasets and dataloaders contained in the **_0_dataset_dataloader_template.ipynb_**

It doesn't require having seen the baselines template, but they are meant to complement each other.  **_In fact they are on purpose almost identical!!!_**

## Set your cuda visible device

**IMPORTANT:** Since we are sharing resources, please make sure that the cuda visible device you put here is the one assigned to your team and your machine.   

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

Here we initalize variables related to Weights and Biases, our online logging system to ensure they are user specific

In [2]:
# Make sure wandb logs are stored in a user-specific directory
# Set writable directories
os.environ["WANDB_DIR"] = "./wandb/wandb_logs"
os.environ["WANDB_CACHE_DIR"] = "./wandb/wandb_cache"
os.environ["WANDB_CONFIG_DIR"] = "./wandb/wandb_config"
# Optional:
os.environ["TMPDIR"] = "./wandb/wandb_tmp"

# Ensure directories exist (optional, wandb usually creates them)
os.makedirs(os.environ["WANDB_DIR"], exist_ok=True)
os.makedirs(os.environ["WANDB_CACHE_DIR"], exist_ok=True)
os.makedirs(os.environ["WANDB_CONFIG_DIR"], exist_ok=True)
os.makedirs(os.environ["TMPDIR"], exist_ok=True)

In [3]:

import sys
from torch.utils.data import DataLoader

import torch
import yaml

import lightning as L
from lightning.pytorch.callbacks import ModelCheckpoint
from lightning.pytorch.loggers import CSVLogger, WandbLogger

# Append base path.  May need to be modified if the folder structure changes.
# It gives the notebook access to the wokshop_infrastructure folder.
sys.path.append("../../")
 
# Append Surya path. May need to be modified if the folder structure changes.
# It gives the notebook access to surya's release code.
sys.path.append("../../Surya")

from surya.utils.data import build_scalers  # Data scaling utilities for Surya stacks
from workshop_infrastructure.utils import apply_peft_lora
torch.set_float32_matmul_precision('medium')



## Download scalers and Weights
Surya input data needs to be scaled properly for the model to work and this cell downloads the scaling information.  In this notebook we also download the model weights for finetuning


- If the cell below fails, try running the provided shell script directly in the terminal.
- Sometimes the download may fail due to network or server issues—if that happens, simply re-run the script a few times until it completes successfully.

In [4]:
!sh download_scalers_and_weights.sh

==> Checking assets directory at: /home/shahbahauddin/surya_workshop/downstream_apps/template/assets
==> Downloading scalers and model weights into: /home/shahbahauddin/surya_workshop/downstream_apps/template/assets
/home/shahbahauddin/surya_workshop/downstream_apps/template/assets
/home/shahbahauddin/surya_workshop/downstream_apps/template/assets
✓ Done. Files are in: /home/shahbahauddin/surya_workshop/downstream_apps/template/assets


## Load configuration

Surya was designed to read a configuration file that defines many aspects of the model
including the data it uses we use this config file to set default values that do not
need to be modified, but also to define values specific to our downstream application

In [5]:
# Configuration paths - modify these if your files are in different locations
config_path = "./configs/config.yaml"

# Load configuration
print("📋 Loading configuration...")
try:
    config = yaml.safe_load(open(config_path, "r"))
    config["data"]["scalers"] = yaml.safe_load(open(config["data"]["scalers_path"], "r"))
    print("✅ Configuration loaded successfully!")
except FileNotFoundError as e:
    print(f"❌ Error: {e}")
    print("Make sure config.yaml exists in your current directory")
    raise

scalers = build_scalers(info=config["data"]["scalers"])

📋 Loading configuration...
✅ Configuration loaded successfully!


## Define Downstream (DS) datasets

This child class takes as input all expected HelioFM parameters, plus additonal parameters relevant to the downstream application.  Here we focus in particular to the DS index and parameters necessary to combine it with the HelioFM index.

Another important component of creating a dataset class for your DS is normalization.  Here we use a log normalization on xray flux that will act as the output target.  Making log10(xray_flux) strictly positive and having 66% of its values between 0 and 1

In this case we will define both a training and a validation dataset using the indices pointed at in the config

**_Important:  In this notebook we sets max_number_of_samples=6 to potentially avoid going through the whole dataset as we explore it.  Keep in mind this for the future in case the database seems smaller than you expect_**


In [6]:
from downstream_apps.template.datasets.template_dataset import FlareDSDataset

In [7]:
from downstream_apps.template.datasets.eve_dataset import EVESpectraDSDataset

# Path to your downstream spectra CSV
ds_spectra_csv_path = "./euv_data/sample_6k_eve_spectra.csv"  # update to your full multi-cycle file

train_dataset = EVESpectraDSDataset(
    #### All these lines are required by the parent HelioNetCDFDataset class
    index_path=config["data"]["train_data_path"],
    time_delta_input_minutes=config["data"]["time_delta_input_minutes"],
    time_delta_target_minutes=config["data"]["time_delta_target_minutes"],
    n_input_timestamps=config["model"]["time_embedding"]["time_dim"],
    rollout_steps=config["rollout_steps"],
    channels=config["data"]["channels"],
    drop_hmi_probability=config["drop_hmi_probability"],
    use_latitude_in_learned_flow=config["use_latitude_in_learned_flow"],
    scalers=scalers,
    phase="train",
    s3_use_simplecache=False,
    s3_cache_dir="/tmp/helio_s3_cache",
    #### Put your downstream (DS) specific parameters below this line
    return_surya_stack=True,
    max_number_of_samples=16,          # set None for full; or an int for debug
    ds_spectra_csv_path=ds_spectra_csv_path,
    ds_time_column="timestamp",
    ds_time_tolerance="10d",
    ds_match_direction="forward",
)

val_dataset = EVESpectraDSDataset(
    #### All these lines are required by the parent HelioNetCDFDataset class
    index_path=config["data"]["valid_data_path"],  # <--- different index path
    time_delta_input_minutes=config["data"]["time_delta_input_minutes"],
    time_delta_target_minutes=config["data"]["time_delta_target_minutes"],
    n_input_timestamps=config["model"]["time_embedding"]["time_dim"],
    rollout_steps=config["rollout_steps"],
    channels=config["data"]["channels"],
    drop_hmi_probability=config["drop_hmi_probability"],
    use_latitude_in_learned_flow=config["use_latitude_in_learned_flow"],
    scalers=scalers,
    phase="val",
    s3_use_simplecache=False,
    s3_cache_dir="/tmp/helio_s3_cache",
    #### Put your downstream (DS) specific parameters below this line
    return_surya_stack=True,
    max_number_of_samples=None,
    ds_spectra_csv_path=ds_spectra_csv_path,
    ds_time_column="timestamp",
    ds_time_tolerance="100d",
    ds_match_direction="forward",
)

len(train_dataset), len(val_dataset)


(16, 3)

In [8]:
from torch.utils.data import Subset
import numpy as np

# Reduce training dataset size for fast debugging
N_DEBUG = 16

# Take the first N samples (deterministic)
train_indices = np.arange(min(N_DEBUG, len(train_dataset)))
train_dataset_debug = Subset(train_dataset, train_indices)

print(f"Training dataset reduced: {len(train_dataset)} → {len(train_dataset_debug)} samples")


Training dataset reduced: 16 → 16 samples


We also intialize separate training and validation dataloaders.   Since we are working in a shared environment.  Using multiprocessing_context="spawn" helps avoid lockups.

In [9]:
batch_size = 2

train_data_loader = DataLoader(
                dataset=train_dataset,
                batch_size=batch_size,
                shuffle=True,
                num_workers=4,
                multiprocessing_context="spawn",
                persistent_workers=True,
                pin_memory=True,
            )

batch_size = 2

val_data_loader = DataLoader(
                dataset=val_dataset,
                batch_size=batch_size,
                num_workers=4,
                multiprocessing_context="spawn",
                persistent_workers=True,
                pin_memory=True,
            )

batch = next(iter(train_data_loader))
batch.keys()

dict_keys(['ts', 'time_delta_input', 'forecast', 'lead_time_delta', 'ds_index'])

## Initialize the HelioSpectformer model

This is the main difference beteween the notebook that trains the simple model and the one that fine-tunes Surya.  

In the case of the finetuning exercise one of the main differences between DS applications is the dimensionality of the output.  In this notebook we use a modified HelioSpectformer that projects into a 1D space. 

**_IMPORTANT: If your DS application is 2D you need to use the HelioSpectformer2D_**

In [10]:
from workshop_infrastructure.models.finetune_models import HelioSpectformer1D
# Infer EVE output dimension (spectrum length)
sample = train_dataset[0]
num_outputs = int(sample["forecast"].shape[-1])
print("EVE num_outputs =", num_outputs)

/opt/anaconda3/envs/surya_ws/lib/python3.12/site-packages/timm/models/layers/__init__.py:49: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


EVE num_outputs = 1343


Now the config file really comes into bear. The Spectformer has a metric ton of hyperparameters

In [11]:
model = HelioSpectformer1D(
    img_size=config["model"]["img_size"],
    patch_size=config["model"]["patch_size"],
    in_chans=config["model"]["in_channels"],
    embed_dim=config["model"]["embed_dim"],
    time_embedding=config["model"]["time_embedding"],
    depth=config["model"]["depth"],
    num_heads=config["model"]["num_heads"],
    mlp_ratio=config["model"]["mlp_ratio"],
    drop_rate=config["model"]["drop_rate"],
    dtype=config["dtype"],
    window_size=config["model"]["window_size"],
    dp_rank=config["model"]["dp_rank"],
    learned_flow=config["model"]["learned_flow"],
    use_latitude_in_learned_flow=config["use_latitude_in_learned_flow"],
    init_weights=config["model"]["init_weights"],
    checkpoint_layers=config["model"]["checkpoint_layers"],
    n_spectral_blocks=config["model"]["spectral_blocks"],
    rpe=config["model"]["rpe"],
    ensemble=config["model"]["ensemble"],
    finetune=config["model"]["finetune"],
    nglo=config["model"]["nglo"],
    # Put finetuning additions below this line
    dropout=config["model"]["dropout"],
    num_penultimate_transformer_layers=0,
    num_penultimate_heads=0,
    num_outputs=num_outputs,
    config=config,
)

## Load model weights

Here we load the pre-trained checkpoint and load the weights.  The exercise of loading follows the idea of us as many of the weights as possible.  This is accomplished through the filtered_checkpoint_state.   It checks to see if the pretrained model's layers match those of your finetuning architecture.   It also checks that all your dimensions across layers check out.   If something does not work those paramameters are left in their random initialization. 

In [12]:
model_state = model.state_dict()
checkpoint_state = torch.load(config["pretrained_path"], weights_only=True, map_location="cpu")
filtered_checkpoint_state = {
    k: v
    for k, v in checkpoint_state.items()
    if k in model_state and v.shape == model_state[k].shape
}

# 2. Load the filtered weights
model_state.update(filtered_checkpoint_state)
model.load_state_dict(model_state, strict=True)

<All keys matched successfully>

## To LoRA or not to Lora

This cell gives you two options.  On the one hand we have the classic freezing of the backbone (the initial layers of the model).   On the other hand we have the use of a LoRA.

LoRas have been a remarkable addition to our arsenal of models.   They have the advantage of keeping pretty much the entire model intact and only add broad modifications to weights as needed.

In [13]:
use_LoRa = True

if use_LoRa:
    model = apply_peft_lora(model, config)
else:
    for name, param in model.named_parameters():
        if "embedding" in name or "backbone" in name:
            param.requires_grad = False
    parameters_with_grads = []
    for name, param in model.named_parameters():
        if param.requires_grad:
            parameters_with_grads.append(name)
    print(
        f"{len(parameters_with_grads)} parameters require gradients: {', '.join(parameters_with_grads)}."
    )

Applying PEFT LoRA with configuration: {'r': 8, 'lora_alpha': 8, 'target_modules': ['q_proj', 'v_proj', 'k_proj', 'out_proj', 'fc1', 'fc2'], 'lora_dropout': 0.1, 'bias': 'none'}
trainable params: 1,024,000 || all params: 362,052,415 || trainable%: 0.28%


We can now test that this model manipulates a batch as expected and returns an estimate of flare intensity as we did for the simple baseline.

We pass the input stack 'ts' to the model to transform it into our regression output.   Note that since this model was trained for a different task, it's likely it won't perform very well.  As with the simple baseline, this only acts as a test that our model forward doesn't have dimension problems.

Dimension problemns are the dominant source of error in this kind of work.

Note that our output has now the size of our batch.

In [14]:
batch = next(iter(train_data_loader))
model.eval()
with torch.no_grad():
    output = model.forward(batch)

# Do NOT print tensors directly in your environment; print shapes only
print("output shape:", tuple(output.shape))
print("target shape:", tuple(batch["forecast"].shape))

output shape: (2, 1343)
target shape: (2, 1343)


## Define your metrics

Metrics are a very important part of training AI models.   They provide your models with the quantitification of error, which in turn shifts the weights towards better pefrorming models.  They also provide a way for you to monitor performance, identify overfitting, and quantify value added. 

We now initialize the metrics class which allows you to control what metrics do you want to use as "loss" (i.e. the metrics that backpropagate through your model) and which ones for monitoring performance.  As with other components, this takes the form of a loaded module that can be later use in a training script

In [15]:
from downstream_apps.template.metrics.template_metrics import FlareMetrics

# import torch

import torch

class EVEMetrics:
    def __init__(self, mode: str, eps: float = 1e-12):
        self.mode = mode
        self.eps = eps

    def _standardize(self, preds: torch.Tensor, target: torch.Tensor):
        preds = preds.float()
        target = target.float()

        while preds.ndim > 2 and preds.shape[-1] == 1:
            preds = preds.squeeze(-1)
        while target.ndim > 2 and target.shape[-1] == 1:
            target = target.squeeze(-1)

        if preds.ndim == 1:
            preds = preds.unsqueeze(-1)
        if target.ndim == 1:
            target = target.unsqueeze(-1)

        if preds.shape != target.shape:
            raise ValueError(f"Shape mismatch: preds {tuple(preds.shape)} vs target {tuple(target.shape)}")
        return preds, target

    def _rrse(self, preds: torch.Tensor, target: torch.Tensor) -> torch.Tensor:
        num = ((preds - target) ** 2).sum()
        mu = target.mean()
        den = ((target - mu) ** 2).sum() + self.eps
        return torch.sqrt(num / den)

    def train_loss(self, preds: torch.Tensor, target: torch.Tensor):
        preds, target = self._standardize(preds, target)
        out = {}
        wts = []
        out["mse"] = torch.nn.functional.mse_loss(preds, target)
        wts.append(1)
        return out, wts

    def train_metrics(self, preds: torch.Tensor, target: torch.Tensor):
        preds, target = self._standardize(preds, target)
        out = {}
        wts = []
        out["rrse"] = self._rrse(preds, target)
        wts.append(1)
        return out, wts

    def val_metrics(self, preds: torch.Tensor, target: torch.Tensor):
        preds, target = self._standardize(preds, target)
        out = {}
        wts = []
        out["mse"] = torch.nn.functional.mse_loss(preds, target)
        wts.append(1)
        out["rrse"] = self._rrse(preds, target)
        wts.append(1)
        return out, wts

    def __call__(self, preds: torch.Tensor, target: torch.Tensor):
        match self.mode.lower():
            case "train_loss":
                return self.train_loss(preds, target)
            case "train_metrics":
                with torch.no_grad():
                    return self.train_metrics(preds, target)
            case "val_metrics":
                with torch.no_grad():
                    return self.val_metrics(preds, target)
            case _:
                raise NotImplementedError(f"{self.mode} is not implemented as a valid metric case.")


In [16]:
train_loss_metrics = EVEMetrics("train_loss")
train_evaluation_metrics = EVEMetrics("train_metrics")
validation_evaluation_metrics = EVEMetrics("val_metrics")

# Suppress tensor printing in your environment; just ensure it runs
_ = train_loss_metrics(output, batch["forecast"])
_ = train_evaluation_metrics(output, batch["forecast"])
_ = validation_evaluation_metrics(output, batch["forecast"])

print("Metrics sanity-check passed.")


Metrics sanity-check passed.


Now they can be evaluated in our model's output and our ground truth.   First the loss that actually will backpropagate, in this case Mean Squared Errror

In [17]:
train_loss_metrics(output, batch["forecast"])

({'mse': tensor(1.8548)}, [1])

Then a training evaluation that will not backpropagate and inform our model, but that we can keep an eye on. Note that reporting lots of metrics during training will slow the training process.  I'm including it her as an example, but oftentimes is better to put the diagnostics only in the validation evaluation metrics.

Here we are caclulating the Root Relative Squared Error https://lightning.ai/docs/torchmetrics/stable/regression/rse.html 

A value below one means the prediction is better than predicting the average.  It is unlikely that this metric will be lower than one with a randomly initialized model

In [18]:
train_evaluation_metrics(output, batch["forecast"])

({'rrse': tensor(1.5067)}, [1])

In the validation evaluation metrics we report both MSE and RRSE

In [19]:
validation_evaluation_metrics(output, batch["forecast"])

({'mse': tensor(1.8548), 'rrse': tensor(1.5067)}, [1, 1])

## Define your PyTorch ligthning module

In this workshop we will use PyTorch lightning to train our models.  PyTorch lighting reduces the amount of code required to implement a training loop in comparison to PyTorch (at the expense of control and versatility).  

Opening the FlareLightningModule shows a simple Lightning model implementation.  It consists of:

- An initialization of the class (metrics, model, and learning rate).
- The forward code that runs evaluation of the model.
- Training and validation steps.
- Configuration of optimizers.

**_Note that it is the same Lightning module we used for the baseline!!_**

In [20]:
from downstream_apps.template.lightning_modules.eve_simple_baseline import EVELightningModule

## Set your global seeds

Since training AI models generally uses stochastic gradient descent, it is a good idea to fix your random seeds so that your training exercise is reproducible.    

In [21]:
L.seed_everything(42, workers=True)

Seed set to 42


42

## Intialize Lightning module

Now we properly initalize the Lightning module to enable training, including passing the dictionary of metrics

In [22]:
metrics = {'train_loss': train_loss_metrics,
           'train_metrics': train_evaluation_metrics,
           'val_metrics': validation_evaluation_metrics}

learning_rate = 1e-3
lit_model = EVELightningModule(model, metrics, lr=learning_rate, batch_size=batch_size)

## Logging

In order to properly compare experiments against each other, it is very useful to log evaluation metrics in a place where they can be compared against other training runs.  In this workshop we will use Weights and Biases (WandB). 

The first time you run WandB in a machine it will ask you to login to WandB.  You should have received an invitation to our project.  In order to login you must:

- Select option 2 (existing account).   In VScode the dialog opens a box at the top of your screen.
- Click on get API Key (this will open a browser).
- Generate API Key.
- Paste it in the dialog box at the top of your VSCode

In [23]:
project_name = "eve_flare_regression"
run_name = "eve_baseline_experiment_1"

wandb_logger = WandbLogger(
    entity="surya_handson",
    project=project_name,
    name=run_name,
    log_model=False,
    save_dir="./wandb/wandb_tmp",
)

csv_logger = CSVLogger("runs", name="simple_flare")

## Initialize trainer

With the loggers done, now the trainer needs to be defined.  The trainer defines several properties of your training run. Here we define:

- The max number of epochs (one epoch represents your model seeing your entire training dataset).
- Define where the training run will take place (auto uses the GPU if possible, if not, CPU).
- The loggers.
- The callbacks (here we save the model with the lowest validation loss).
- Logging frequency (because we are working with a small dataset it needs to be small).


**Note that in this notebook we also set a mixed precision to reduce the model's footprint in memory.**

In [24]:
max_epochs = 2

# -------------------------------------------------------------------------
# Trainer
# -------------------------------------------------------------------------
trainer = L.Trainer(
    max_epochs=max_epochs,
    accelerator="auto",
    devices="auto",
    precision="bf16-mixed", 
    logger=[wandb_logger, csv_logger],
    callbacks=[
        ModelCheckpoint(
            monitor="val_loss",
            mode="min",
            save_top_k=1,
        )
    ],
    log_every_n_steps=2,
)

Using bfloat16 Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores


## Fit the model

Finally we fit the model.  We pass the Lighting module, and our dataloaders.

In [25]:
trainer.fit(lit_model, train_data_loader, val_data_loader)

wandb: WARNING The anonymous setting has no effect and will be removed in a future version.
wandb: Currently logged in as: shah-bahauddin (shah-bahauddin-university-of-colorado-boulder) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [2]
/opt/anaconda3/envs/surya_ws/lib/python3.12/site-packages/lightning/pytorch/utilities/model_summary/model_summary.py:242: Precision bf16-mixed is not supported by the model summary.  Estimated model size in MB will not be accurate. Using 32 bits instead.

  | Name  | Type      | Params | Mode | FLOPs
---------------------------------------------------
0 | model | PeftModel | 362 M  | eval | 0    
---------------------------------------------------
1.0 M     Trainable params
361 M     Non-trainable params
362 M     Total params
1,448.210 Total estimated model params size (MB)
0         Modules in train mode
369       Modules in eval mode
0         Total Flops


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/opt/anaconda3/envs/surya_ws/lib/python3.12/site-packages/lightning/pytorch/loops/fit_loop.py:534: Found 369 module(s) in eval mode at the start of training. This may lead to unexpected behavior during training. If this is intentional, you can ignore this warning.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=2` reached.


## Conclusion

With this we have now integrated our dataset, dataloaders, metrics, and DS into an end-2-end training loop and we are ready to experiment!